<a href="https://colab.research.google.com/github/Sagnik-Nandi/PDFQueryBot---Chatbot-over-PDFs-using-RAG/blob/main/assnmt%201%20-%20Digit%20recognition%20using%20Pytorch/digit_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Dataset

Before running, YOU need to create a 'data' directory in the current directory.

## Imports and Data Exploration
- Importing pytorch and other libs
- Create dataset and dataloader instances


In [116]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [117]:
# Loading the dataset
train=datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test=datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [118]:
# Some idea about the dataset
print(train)
print(test)
print(train[0][0].shape)
print(train[0][1])

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()
torch.Size([1, 28, 28])
5


## Train and validation split   
- create dataloader instances for each dataset
- Hyperparameter used:
1. Batch_size = 8   
Reducing batch_size significantly improved accuracy, but too small batch_size makes the model computationally large and unstable

In [119]:
trainAll=train
train, eval=random_split(trainAll, [50000, 10000])

# Hyperparameter: Batch_size
batch_size=8

# Dataloader for : train, eval, test
train_loader=torch.utils.data.DataLoader(
    train,
    batch_size=batch_size,
    shuffle=True,
  )
eval_loader=torch.utils.data.DataLoader(
    eval,
    batch_size=batch_size,
    shuffle=True,
  )
test_loader=torch.utils.data.DataLoader(
    test,
    batch_size=batch_size,
    shuffle=True,
  )


In [120]:
# Shows sample images from the dataset using plt.imshow
def sampleData(data):
  figure = plt.figure(figsize=(8, 8))
  cols, rows = 3, 3
  for i in range(1, cols * rows + 1):
      sample_idx = torch.randint(len(data), size=(1,)).item()
      img, label = data[sample_idx]
      figure.add_subplot(rows, cols, i)
      plt.title(label)
      plt.axis("off")
      plt.imshow(img.squeeze(), cmap="gray")
  plt.show()
# sampleData(train)

# Make a Neural Network model
- create model and forward function   
- Hyperparameters used:
2. no. of hidden layers = 1   
increasing hidden layers does not improve accuracy
2. no. of units in layer1 = 32   
changing number of units does not affect accuracy

In [121]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() # flattens 28*28 2d array to 1d vector
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            # nn.Linear(32, 32),
            # nn.ReLU(),
            # nn.Linear(512, 512),
            # nn.ReLU(),
            nn.Linear(32, 10), # output layer has 10 values, for i in (0, 9) : Pr(input==i)
        )

    def forward(self, x): # forward propagation function
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model=NeuralNetwork()

## Training and testing
 - Hyperparameters used:
4. learning_rate = 0.001   
  Critical parameter
5. epoch = 10 (for testing and tuning purposes, set to 5)   
  Critical parameter, increasing no. of epochs improved accuracy. But too large no. of epochs can be computaionally large and may lead to overfitting

- Loss Function   
Cross Entropy function is generally suitable in discrete-valued prediction problem, such as this one

- Optimizer   
Optimizer compiles the backward propagation step and update the parameters based on different algorithms, such as Gradient Descent   
Tested with different ones like SGD(Stochastic Gradient Descent), ADAM() etc.

In [122]:
# Hyperparameters:
learning_rate = 1e-3
# batch_size = 8
epochs = 5

loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [123]:
def train_loop(dataloader, model, loss_fn, optimizer, verbose=True):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation, update parameters and reset all the gradients to zero
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Show progress report :D
        if verbose and batch % 500 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"Training Loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def eval_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    valid_loss, correct = 0, 0

    # torch.no_grad() ensures that no gradients are computed during eval mode
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            valid_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    valid_loss /= num_batches
    correct /= size
    print(f"Validation Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {valid_loss:>8f} \n")
    return valid_loss

In [124]:
# REF: https://www.geeksforgeeks.org/training-neural-networks-with-validation-using-pytorch/

def fit(model, epochs, train_loader, eval_loader, loss_fn, optimizer, verbose=True):
  min_valid_loss = np.inf
  for t in range(epochs):
      print(f"Epoch {t+1}\n-------------------------------")
      train_loop(train_loader, model, loss_fn, optimizer, verbose)
      valid_loss = eval_loop(eval_loader, model, loss_fn)

      # Saving parameters
      if min_valid_loss > valid_loss:
          print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \n Saving The Model\n')
          min_valid_loss = valid_loss
          torch.save(model.state_dict(), 'saved_model.pth')
  print("Done!")

fit(model, epochs, train_loader, eval_loader, loss_fn, optimizer, verbose=False)


Epoch 1
-------------------------------
Validation Error: 
 Accuracy: 93.1%, Avg loss: 0.232022 

Validation Loss Decreased(inf--->0.232022) 
 Saving The Model

Epoch 2
-------------------------------
Validation Error: 
 Accuracy: 94.4%, Avg loss: 0.189673 

Validation Loss Decreased(0.232022--->0.189673) 
 Saving The Model

Epoch 3
-------------------------------
Validation Error: 
 Accuracy: 95.3%, Avg loss: 0.164218 

Validation Loss Decreased(0.189673--->0.164218) 
 Saving The Model

Epoch 4
-------------------------------
Validation Error: 
 Accuracy: 95.4%, Avg loss: 0.159925 

Validation Loss Decreased(0.164218--->0.159925) 
 Saving The Model

Epoch 5
-------------------------------
Validation Error: 
 Accuracy: 95.6%, Avg loss: 0.148704 

Validation Loss Decreased(0.159925--->0.148704) 
 Saving The Model

Done!


In [125]:
def score(model, test_loader, loss_fn):
  model.load_state_dict(torch.load('saved_model.pth'))
  model.eval()
  correct=0
  size=len(test_loader.dataset)

  for X, y in test_loader:
    pred = model(X)
    correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  correct /= size
  print(f"Test score: \n Accuracy: {(100*correct):>0.1f}%")

score(model, test_loader, loss_fn)

Test score: 
 Accuracy: 96.2%
